In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
path= "data/Crop_recommendation.csv"
data = pd.read_csv(path)
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [20]:
data.isnull().sum()
data.shape
data.columns
data.value_counts
data["label"].unique()
data.dtypes





N                int64
P                int64
K                int64
temperature    float64
humidity       float64
ph             float64
rainfall       float64
label           object
dtype: object

In [2]:
#Visualizing


data_summary = data.pivot_table(data,index=['label'],aggfunc='mean')
data_summary.head()

# @title N

from matplotlib import pyplot as plt
data['N'].plot(kind='hist', bins=20, title='N')
plt.gca().spines[['top', 'right',]].set_visible(False)



,K,N,P,humidity,ph,rainfall,temperature
label,,,,,,,
apple,199.89,20.80,134.22,92.333383,5.929663,112.654779,22.630942
banana,50.05,100.23,82.01,80.358123,5.983893,104.626980,27.376798
blackgram,19.24,40.02,67.47,65.118426,7.133952,67.884151,29.973340
chickpea,79.92,40.09,67.79,16.860439,7.336957,80.058977,18.872847
coconut,30.59,21.98,16.93,94.844272,5.976562,175.686646,27.409892


In [29]:
#anlyzing phosphorus


In [2]:
#Feature selection

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
features = ["N","P","K","temperature","humidity","ph","rainfall"]
X = data[features]
y = data["label"]
X_train,X_test,y_train,y_test =  train_test_split(X,y,random_state=21,test_size=0.2)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)




In [3]:
#Model selection 
#Knarest neighbours

from sklearn.neighbors import KNeighborsClassifier

knn_model =  KNeighborsClassifier(n_neighbors = 5, metric = "minkowski", p=2)
knn_model.fit(X_train_scaled,y_train)

knn_model.score(X_test_scaled,y_test)

0.9772727272727273

In [4]:
#Support vector
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Create the SVC classifier
svm_model = SVC()

# Create GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, verbose=1)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

# Get the best estimator
best_svc = grid_search.best_estimator_

# Evaluate the best estimator on the test set
accuracy = best_svc.score(X_test_scaled, y_test)
print("Accuracy on Test Set:", accuracy)






Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best Parameters: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Accuracy on Test Set: 0.9840909090909091


In [4]:
#Final svm model
from sklearn.svm import SVC

best_params = {'C': 10, 'gamma': 1, 'kernel': 'rbf'}

# Create SVM model with best parameters
svm_rbf_model = SVC(**best_params)
svm_rbf_model.fit(X_train_scaled,y_train)
svm_rbf_model.score(X_test_scaled,y_test)

0.9840909090909091

In [5]:
#Decision Tree model

from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)
decision_tree_model.score(X_test,y_test)



0.9886363636363636

In [12]:
#Random Forest Model
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'max_depth': [2, 4, 7, 10,14,21],
    'n_estimators': [100, 200, 300, 450],
    'random_state': [7,42,57,83,100]
}

random_forest_model = RandomForestClassifier()
grid_search = GridSearchCV(random_forest_model, param_grid, cv=10, verbose=1,n_jobs=-1)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

# Get the best estimator
best_random_forest = grid_search.best_estimator_

# Evaluate the best estimator on the test set
accuracy = best_random_forest.score(X_test_scaled, y_test)
print("Accuracy on Test Set:", accuracy)


#random_forest_model.fit(X_train_scaled,y_train)
#random_forest_model.score(X_test_scaled,y_test)

Fitting 10 folds for each of 120 candidates, totalling 1200 fits
Best Parameters: {'max_depth': 10, 'n_estimators': 300, 'random_state': 57}
Accuracy on Test Set: 0.9863636363636363


In [6]:
#Final Random Forest
from joblib import dump
from sklearn.ensemble import RandomForestClassifier
Best_Parameters= {'max_depth': 10, 'n_estimators': 300, 'random_state': 57}
random_forest_model = RandomForestClassifier(**Best_Parameters)
random_forest_model.fit(X_train,y_train)
random_forest_model.score(X_test,y_test)
#dump(random_forest_model,"random_forest_model.joblib")


0.9931818181818182

In [7]:
#Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

gradient_boosting_model = GradientBoostingClassifier()
gradient_boosting_model.fit(X_train,y_train)
gradient_boosting_model.score(X_test,y_test)

0.9863636363636363

In [13]:
from joblib import load
import numpy as np
model = load("random_forest_model.joblib")
input_array = np.array([199.89,20.80,134.22,92.333383,5.9296663,112.564779,22.630942])
input_array.reshape(-1,1)
model.predict([input_array])

c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['muskmelon'], dtype=object)

In [20]:
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [10]:
#Models 
from joblib import dump

dump(knn_model,"knn_model.joblib")
dump(svm_rbf_model,"svm_rbf_model.joblib")
dump(decision_tree_model,"decision_tree_model.joblib")
dump(gradient_boosting_model,"gradient_boosting_model.joblib")




['gradient_boosting_model.joblib']

In [8]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
final_estimator = LogisticRegression()
stacking_clf = StackingClassifier(estimators=[
    ('knn', knn_model),
    ('svm', svm_rbf_model),
    ('tree', decision_tree_model),
    ('rf', random_forest_model),
    ('gb', gradient_boosting_model)
])
stacking_clf.fit(X_train, y_train)




c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('knn', KNeighborsClassifier()),
                               ('svm', SVC(C=10, gamma=1)),
                               ('tree',
                                DecisionTreeClassifier(random_state=42)),
                               ('rf',
                                RandomForestClassifier(max_depth=10,
                                                       n_estimators=300,
                                                       random_state=57)),
                               ('gb', GradientBoostingClassifier())])

In [15]:
import numpy as np
predictions = stacking_clf.predict_proba(X_test)
crops = stacking_clf.predict(X_test)
# Define the number of recommendations (top 'n' varieties)
top_n = 3

# Get the top 'n' indices with highest probabilities for each data point
top_n_indices = np.argsort(predictions, axis=1)[:, -top_n:]
variety_names = []

# Extract the corresponding probabilities (compatibility scores)
top_n_scores = predictions[np.arange(len(predictions))[:, None], top_n_indices]

# Loop through each data point and its recommendations
for i in range(len(X_test)):
    print(f"Data point {i+1}:")
    for j in range(top_n):
        variety_index = top_n_indices[i, j]
        score = top_n_scores[i, j]
        # Replace 'variety_names' with your actual list of crop variety names
        print(f"\t- Recommended variety: (Score: {score:.4f})")



Data point 1:
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.9790)
Data point 2:
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.9792)
Data point 3:
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.9788)
Data point 4:
	- Recommended variety: (Score: 0.0013)
	- Recommended variety: (Score: 0.0013)
	- Recommended variety: (Score: 0.9807)
Data point 5:
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.0014)
	- Recommended variety: (Score: 0.9808)
Data point 6:
	- Recommended variety: (Score: 0.0017)
	- Recommended variety: (Score: 0.0017)
	- Recommended variety: (Score: 0.9745)
Data point 7:
	- Recommended variety: (Score: 0.0008)
	- Recommended variety: (Score: 0.0008)
	- Recommended variety: (Score: 0.9888)
Data point 8:
	- Recommended variety: (Score: 0.0010)
	- Recom

In [22]:
crop=stacking_clf.predict([[199.89,20.80,134.22,92.333383,5.9296663,112.564779,22.630942]])
compactibilty=stacking_clf.predict_proba([[199.89,20.80,134.22,92.333383,5.9296663,112.564779,22.630942]])
print(crop)


c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, 

['muskmelon']


c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [25]:
print(f"{crop},{compactibilty.max()*10}")


['muskmelon'],8.26391995280632


In [27]:

# Assuming your test data is loaded into a pandas dataframe named 'X_test'

# Get predictions (compatibility scores)
compatibility = stacking_clf.predict_proba(X_test)

# Get predicted crop varieties (class labels)
crop = stacking_clf.predict(X_test)

# Access the corresponding crop variety names from your label list (assuming it's stored in 'labels')
#predicted_varieties = labels[crop]

# Loop through each data point in the test set and print results
for i in range(len(X_test)):
    print(f"Data point {i+1}:")
    print(f"\t- Predicted Crop Variety: {crop[i]}")
    print(f"\t- Compatibility Score: {compatibility[i, np.argmax(compatibility[i])]:.4f}")

Data point 1:
	- Predicted Crop Variety: pomegranate
	- Compatibility Score: 0.9790
Data point 2:
	- Predicted Crop Variety: grapes
	- Compatibility Score: 0.9792
Data point 3:
	- Predicted Crop Variety: grapes
	- Compatibility Score: 0.9788
Data point 4:
	- Predicted Crop Variety: apple
	- Compatibility Score: 0.9807
Data point 5:
	- Predicted Crop Variety: mothbeans
	- Compatibility Score: 0.9808
Data point 6:
	- Predicted Crop Variety: watermelon
	- Compatibility Score: 0.9745
Data point 7:
	- Predicted Crop Variety: mothbeans
	- Compatibility Score: 0.9888
Data point 8:
	- Predicted Crop Variety: lentil
	- Compatibility Score: 0.9838
Data point 9:
	- Predicted Crop Variety: cotton
	- Compatibility Score: 0.9758
Data point 10:
	- Predicted Crop Variety: blackgram
	- Compatibility Score: 0.9177
Data point 11:
	- Predicted Crop Variety: pigeonpeas
	- Compatibility Score: 0.9813
Data point 12:
	- Predicted Crop Variety: muskmelon
	- Compatibility Score: 0.9860
Data point 13:
	- Predict

In [9]:
dump(stacking_clf, "stacking_classifier_model.pkl")

['stacking_classifier_model.pkl']

In [10]:
dump(stacking_clf, "stacking_classifier_model.joblib")

['stacking_classifier_model.joblib']

In [16]:
from joblib import load
stacked_model = load("stacking_classifier_model.joblib")
print(stacked_model.predict([[90,26,26,26,56,6.5,182]]))
print(stacked_model.predict_proba([[90,26,26,26,56,6.5,182]]))

['coffee']
[[7.52822589e-04 8.12682955e-04 6.19590299e-04 7.40531375e-04
  7.64994807e-04 9.83003576e-01 5.75720429e-04 7.69784695e-04
  7.18879183e-04 7.42847413e-04 1.04832853e-03 1.24056482e-03
  8.11144124e-04 9.18511163e-04 7.40532114e-04 1.12191232e-03
  7.51748213e-04 9.14707776e-04 7.66817728e-04 7.52243901e-04
  8.24081037e-04 6.07978752e-04]]


c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, 